In [1]:
import pyspark
from pyspark.sql import SparkSession

# Создание сессии Spark
spark = SparkSession.builder.appName("example").getOrCreate()

# Получение URL веб-интерфейса Spark UI
ui_url = spark.sparkContext.uiWebUrl
print("URL веб-интерфейса Spark UI:", ui_url)


URL веб-интерфейса Spark UI: http://EPUSPRIW0A14.attlocal.net:4041


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.createOrReplaceTempView ('green')

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [9]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-06 07:00:00| 241|            169.16|             6|
|2020-01-19 19:00:00|  41| 457.2400000000003|            43|
|2020-01-23 23:00:00| 129| 304.1300000000001|            22|
|2020-01-28 15:00:00| 129|            208.23|             9|
|2020-01-02 08:00:00|  74|1187.4199999999987|            96|
|2020-01-06 16:00:00| 129|250.42000000000002|            15|
|2020-01-07 10:00:00| 171|             38.98|             2|
|2020-01-09 15:00:00| 260| 242.6800000000001|            19|
|2020-01-16 20:00:00|  82| 424.1900000000001|            27|
|2020-01-21 16:00:00|  33| 917.3599999999994|            38|
|2020-01-08 17:00:00| 140|            313.81|             8|
|2020-01-22 08:00:00| 244| 285.3999999999999|            14|
|2020-01-30 14:00:00|  74| 738.6799999999996|            49|
|2020-01-03 07:00:00|  7

In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [11]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [19]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [20]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [21]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000001|                   57|
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|  56|             99.69|                   3|              18.1|                    2|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|199.49000000000004|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  74|317.09000000000015|                  24| 586.2100000000002|                   47|
|2020-01-01 00:00:00|  88|  

In [22]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [23]:
df_join = spark.read.parquet('data/report/revenue/total')

In [26]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  12|              NULL|                NULL|             107.0|                    6|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  25|             531.0|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  29|              61.3|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  47|  

In [28]:
df_zones = spark.read.parquet('zones/')

Py4JJavaError: An error occurred while calling o80.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 147) (EPUSPRIW0A14.attlocal.net executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:383)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.s

In [29]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

NameError: name 'df_zones' is not defined

In [30]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

NameError: name 'df_result' is not defined